In [39]:
#https://medium.com/data-science/building-knowledge-graphs-with-llm-graph-transformer-a91045c49b59

In [47]:
import os
from typing import Dict, Any, Optional
from dotenv import load_dotenv
from neo4j import GraphDatabase

# LangChain imports
from langchain_neo4j import Neo4jVector, Neo4jGraph
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

In [21]:
# Neo4j Configuration
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7689")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password123")

# Configuration de l'embedding (doit correspondre à celle utilisée dans 003_index_content.py)
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME", "Qwen/Qwen3-Embedding-0.6B")
EMBEDDING_MODEL_KWARGS = {"device": "cpu"}  # Utiliser "cuda" si GPU disponible
EMBED_ENCODE_KWARGS = {"normalize_embeddings": True}  # Normaliser pour la similarité cosinus

# Configuration de l'index vectoriel
VECTOR_INDEX_NAME = os.getenv("VECTOR_INDEX_NAME", "content_vector_qwen_index")
EMBEDDING_NODE_PROPERTY = "embedding_qwen"  # Nom de la propriété où l'embedding est stocké

# Nombre de résultats par défaut
DEFAULT_K = 20

In [22]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USER, password=NEO4J_PASSWORD, enhanced_schema=True)

In [23]:
os.environ["OPENAI_API_KEY"] = "no_need"
os.environ["OPENAI_BASE_URL"] = "https://vlm.smart-classifier.prod.bmdspapp.de/v1/"
llm = ChatOpenAI(model='qwen')

In [35]:
no_schema = LLMGraphTransformer(llm=llm, node_properties=True, relationship_properties=True)

In [52]:
results = graph.query("MATCH (n:Content) RETURN n LIMIT 2")
results, 
len(results)
results[0]['n']['chunk_content']

'Sofern während der Beurlaubung Beiträge zu entrichten sind, ist deren Zahlung\nVoraussetzung für die Beurlaubung.\n\n(3) Ein Antrag auf Beurlaubung ist grundsätzlich innerhalb der von der Fachhochschule für die\nRückmeldung festgesetzten Frist zu stellen. Ausnahmen sind zulässig, wenn die\nVoraussetzungen für die Beurlaubung erst zu einem späteren Zeitpunkt eintreten; über\nAusnahmen entscheidet die Hochschulverwaltung.\n\n(4) Eine Beurlaubung ist nur möglich, sofern die beziehungsweise der Studierende durch\neinen der oben genannten Beurlaubungsgründe mindestens für die Hälfte des Semesters\nan der Erbringung von Studienleistungen beziehungsweise am Besuch von\nLehrveranstaltungen gehindert ist.\nDie Beurlaubung erfolgt jeweils für die Dauer eines vollen Semesters.\nEine Beurlaubung über ein Semester hinaus ist nur bei besonders nachzuweisenden\nGründen zulässig; sie erfolgt, wenn das Fortbestehen des Beurlaubungsgrundes für jedes\nSemester im Zeitraum der Rückmeldung unter Beifügung

In [53]:
documents = [Document(page_content=r['n']['chunk_content']) for r in results]

In [54]:
documents

[Document(metadata={}, page_content='Sofern während der Beurlaubung Beiträge zu entrichten sind, ist deren Zahlung\nVoraussetzung für die Beurlaubung.\n\n(3) Ein Antrag auf Beurlaubung ist grundsätzlich innerhalb der von der Fachhochschule für die\nRückmeldung festgesetzten Frist zu stellen. Ausnahmen sind zulässig, wenn die\nVoraussetzungen für die Beurlaubung erst zu einem späteren Zeitpunkt eintreten; über\nAusnahmen entscheidet die Hochschulverwaltung.\n\n(4) Eine Beurlaubung ist nur möglich, sofern die beziehungsweise der Studierende durch\neinen der oben genannten Beurlaubungsgründe mindestens für die Hälfte des Semesters\nan der Erbringung von Studienleistungen beziehungsweise am Besuch von\nLehrveranstaltungen gehindert ist.\nDie Beurlaubung erfolgt jeweils für die Dauer eines vollen Semesters.\nEine Beurlaubung über ein Semester hinaus ist nur bei besonders nachzuweisenden\nGründen zulässig; sie erfolgt, wenn das Fortbestehen des Beurlaubungsgrundes für jedes\nSemester im Zeit

In [55]:
data = await no_schema.aconvert_to_graph_documents(documents)

In [56]:
data

[GraphDocument(nodes=[Node(id='Beurlaubung', type='Concept', properties={}), Node(id='Beiträge', type='Concept', properties={}), Node(id='Fachhochschule', type='Institution', properties={}), Node(id='Studierende', type='Person', properties={}), Node(id='Semester', type='Concept', properties={}), Node(id='Rückmeldung', type='Concept', properties={}), Node(id='Hochschulverwaltung', type='Institution', properties={}), Node(id='Prüfungsordnungen', type='Concept', properties={}), Node(id='Studienleistungen', type='Concept', properties={}), Node(id='Lehrveranstaltungen', type='Concept', properties={}), Node(id='Kinderbetreuung', type='Concept', properties={}), Node(id='Hochschulgesetz', type='Document', properties={}), Node(id='Regelstudienzeit', type='Concept', properties={}), Node(id='Exmatrikulation', type='Concept', properties={}), Node(id='Studiengangwechsel', type='Concept', properties={}), Node(id='Studienformwechsel', type='Concept', properties={}), Node(id='Hochschulwechsel', type='